<div style="text-align: center;">
    <img src="dermatology_icon.png" alt=Dermatologyo" title=Dermatologyo" width="150" />
</div>


<a id='Data-Exploration'></a>
# 1. Data Exploration

In [1]:
# Import the necessary libraries and configurations
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import sklearn
import datetime
import os
from tensorflow import keras
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Input, concatenate, GlobalAveragePooling2D, BatchNormalization
from keras.applications.densenet import DenseNet201, DenseNet121
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
from keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import clone_model
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau
from tensorflow.keras import backend as K
from sklearn.metrics import f1_score
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalMaxPooling2D, Dense, Dropout
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, SGD, Adagrad
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, top_k_accuracy_score

In [2]:
# Reading the dataset
dataset = pd.read_csv('fitzpatrick17k_processed.csv')

In [3]:
# Reading the dataset containing the pictures without body hair
dataset = dataset[dataset['image_path'].notna()]
dataset['image_path'] = 'images_hair_removed/' + dataset['url_alphanum'].astype(str)

<a id='Data-Preprocessing'></a>
# 2. Data Preprocessing

In [4]:
# Creating a copy of the dataset for preprocessing purposes
df_processed = dataset.copy()

In [5]:
# Drop the records without a url, since the model won't be able to learn from records without pictures
df_processed = df_processed[dataset['image_path'].notna()]

In [6]:
# Drop the irrelevant features for the deep learning model
df_processed.drop(['md5hash', 'qc', 'url_alphanum', 'three_partition_label'], axis=1, inplace=True)

In [7]:
# Remove the records where both 'fitzpatrick_scale' and 'fitzpatrick_centaur' are -1
df_processed = df_processed[~((df_processed['fitzpatrick_scale'] == -1) & (df_processed['fitzpatrick_centaur'] == -1))]

In [8]:
# Calculate an average based on both fitzpatrick columns
def calculate_fitzpatrick_average(row):
    if row['fitzpatrick_scale'] == -1 and row['fitzpatrick_centaur'] != -1:
        return row['fitzpatrick_centaur']
    elif row['fitzpatrick_centaur'] == -1 and row['fitzpatrick_scale'] != -1:
        return row['fitzpatrick_scale']
    else:
        return (row['fitzpatrick_scale'] + row['fitzpatrick_centaur']) / 2

# Apply the function to each row
df_processed['fitzpatrick_average'] = df_processed.apply(calculate_fitzpatrick_average, axis=1)

In [9]:
print(f'The dataset is now composed of {df_processed.shape[0]} rows and {df_processed.shape[1]} columns')

The dataset is now composed of 16236 rows and 7 columns


In [10]:
# Inspecting the first 3 records of the dataset
df_processed.head(3)

,fitzpatrick_scale,fitzpatrick_centaur,label,nine_partition_label,url,image_path,fitzpatrick_average
0,3,3,drug induced pigmentary changes,inflammatory,https://www.dermaamin.com/site/images/clinical...,images_hair_removed/httpwwwdermaamincomsiteima...,3.0
1,1,1,photodermatoses,inflammatory,https://www.dermaamin.com/site/images/clinical...,images_hair_removed/httpwwwdermaamincomsiteima...,1.0
2,2,3,dermatofibroma,benign dermal,https://www.dermaamin.com/site/images/clinical...,images_hair_removed/httpwwwdermaamincomsiteima...,2.5


<a id='RGB-Conversion'></a>
### 2.1 Convert Images to an RGB array

In [11]:
def load_and_preprocess_image(path, target_size=(128, 128)):
    try:
        image = Image.open(path)                                # Open the image from the local path
        image = image.convert('RGB')                            # Convert to RGB
        image = np.array(image)                                 # Convert the image to a numpy array
        image = tf.convert_to_tensor(image, dtype=tf.float16)   # Convert to TensorFlow tensor
        image = tf.image.resize(image, target_size)             # Resize the image to the target size
        image /= 255.0                                         # Normalize the image values to be between 0 and 1
        image = np.expand_dims(image, axis=0)                   # Add a batch dimension
        return image
    except Exception as e:
        print(f"An error occurred: {e}")
        print(f"Path: {path}")
        return None

In [12]:
# Create a new column containing the RGB matrix of the pictures
df_processed['rgb'] = df_processed['image_path'].apply(lambda x: load_and_preprocess_image(x))

In [13]:
# Initialize an empty list to store the images' data
images = list()

# Iterate over each row in the dataframe
for i in range(df_processed.shape[0]):
    # Append the image data (from 'rgb' column) of each row to the 'images' list
    # 'iloc[i]' is used to access the i-th row of the dataframe.
    images.append(df_processed['rgb'].iloc[i])

# Convert the list of images into a NumPy array
images = np.array(images)

In [14]:
# Reshape the array to fit the model input:
# 'np.squeeze' is used to remove axes of length one from each sub-array within 'images'
# This operation is done because the model will require input without these single dimensions
# The loop iterates over each sub-array ('subarr') in 'images'
reshaped_image = [np.squeeze(subarr) for subarr in images]

# Convert the list of reshaped arrays back into a single NumPy array because the model expects the input to be a NumPy array rather than a list of arrays
images = np.array(reshaped_image)

In [15]:
# Confirming the shape of the images array
images.shape

(16236, 128, 128, 3)

In [16]:
# Encode labels:
# Initialize a LabelEncoder instance that will be used to convert categorical text labels into a numeric format, making them readable for the models
label_encoder = LabelEncoder()

# Transform the labels into a numeric format:
# 1. Extract the 'label' column from 'df_processed' dataframe and convert it to a NumPy array
# 2. Use the 'fit_transform' method of LabelEncoder to fit the label encoder and return encoded labels
#    This method first fits the label encoder to the data (learning the unique labels) and then transforms the labels to numeric values
# The transformed labels are stored in the 'labels' variable
labels = label_encoder.fit_transform(np.array(df_processed['label']))

<a id='Train-validation-test-split'></a>
### 2.2 Split the data into train, validation, test

In [17]:
# Check the current dataframe's columns
df_processed.columns

Index(['fitzpatrick_scale', 'fitzpatrick_centaur', 'label',
       'nine_partition_label', 'url', 'image_path', 'fitzpatrick_average',
       'rgb'],
      dtype='object')

In [18]:
# Separate the dataset into numerical, categorical and images
df_numerical = df_processed[['fitzpatrick_average']]
df_categorical = df_processed[['nine_partition_label']]

# First split: separate out the training data (60%), while preserving the distribution of label classes in the original dataset, ensuring representativity
X_train_image, X_temp_image, X_train_numerical, X_temp_numerical, X_train_categorical, X_temp_categorical, y_train, y_temp = train_test_split(images, df_numerical, df_categorical, labels, test_size=0.4, stratify=labels, random_state=42)

# Second split: divide the remaining 40% into validation and test sets (50% each of the remaining data), 
# while preserving the distribution of label classes in the original dataset, ensuring representativity
X_val_image, X_test_image, X_val_numerical, X_test_numerical, X_val_categorical, X_test_categorical, y_val, y_test = train_test_split(X_temp_image, X_temp_numerical, X_temp_categorical, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

# This results in 60% training, 20% validation, and 20% test
print("Training set:", X_train_numerical.shape, y_train.shape)
print("Validation set:", X_val_numerical.shape, y_val.shape)
print("Test set:", X_test_numerical.shape, y_test.shape)

Training set: (9741, 1) (9741,)
Validation set: (3247, 1) (3247,)
Test set: (3248, 1) (3248,)


<a id='Normalization'></a>
### 2.3 Normalization

In [19]:
# Initialize the MinMaxScaler object
scaler = MinMaxScaler()

# Fit the scaler exclusively on the training data, to avoid Data Leakage
scaler.fit(X_train_numerical)

# Apply the transformation to the training, validation, and test data
X_train_numerical = scaler.transform(X_train_numerical)
X_val_numerical = scaler.transform(X_val_numerical)
X_test_numerical = scaler.transform(X_test_numerical)

# Now the 'fitzpatrick_average' column is scaled from 0 to 1 in all datasets

In [20]:
# Confirm the success of the normalization in the training dataset
print(f'Minimum fitzpatrick_average in train {X_train_numerical.min()}')
print(f'Maximum fitzpatrick_average in train {X_train_numerical.max()}')

Minimum fitzpatrick_average in train 0.0
Maximum fitzpatrick_average in train 1.0000000000000002


In [21]:
# Confirm the success of the normalization in the validation dataset
print(f'Minimum fitzpatrick_average in validation {X_val_numerical.min()}')
print(f'Maximum fitzpatrick_average in validation {X_val_numerical.max()}')

Minimum fitzpatrick_average in validation 0.0
Maximum fitzpatrick_average in validation 1.0000000000000002


In [22]:
# Confirm the success of the normalization in the testing dataset
print(f'Minimum fitzpatrick_average in test {X_test_numerical.min()}')
print(f'Maximum fitzpatrick_average in test {X_test_numerical.max()}')

Minimum fitzpatrick_average in test 0.0
Maximum fitzpatrick_average in test 1.0000000000000002


<a id='One-Hot-Encoding'></a>
### 2.4 One-hot encoding

In [23]:
# Initialize the OneHotEncoder
# Set `handle_unknown='ignore` to ignore categories that weren't seen during `fit`
encoder = OneHotEncoder(handle_unknown='ignore')

# Fit the encoder exclusively on the training data, to avoid Data Leakage
encoder.fit(X_train_categorical)

# Transform the training data
train_encoded = encoder.transform(X_train_categorical)

# Transform the validation and test data
val_encoded = encoder.transform(X_val_categorical)
test_encoded = encoder.transform(X_test_categorical)

# The output of the encoding is a sparse matrix
# Convert it back to a DataFrame
X_train_one_hot = pd.DataFrame(train_encoded.toarray(), columns=encoder.get_feature_names_out())
X_val_one_hot = pd.DataFrame(val_encoded.toarray(), columns=encoder.get_feature_names_out())
X_test_one_hot = pd.DataFrame(test_encoded.toarray(), columns=encoder.get_feature_names_out())

In [24]:
# Confirm the success of the one-hot encoding procedure on the training dataset
X_train_one_hot.head(3)

,nine_partition_label_benign dermal,nine_partition_label_benign epidermal,nine_partition_label_benign melanocyte,nine_partition_label_genodermatoses,nine_partition_label_inflammatory,nine_partition_label_malignant cutaneous lymphoma,nine_partition_label_malignant dermal,nine_partition_label_malignant epidermal,nine_partition_label_malignant melanoma
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [25]:
# Confirm the success of the one-hot encoding procedure on the validation dataset
X_val_one_hot.head(3)

,nine_partition_label_benign dermal,nine_partition_label_benign epidermal,nine_partition_label_benign melanocyte,nine_partition_label_genodermatoses,nine_partition_label_inflammatory,nine_partition_label_malignant cutaneous lymphoma,nine_partition_label_malignant dermal,nine_partition_label_malignant epidermal,nine_partition_label_malignant melanoma
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [26]:
# Confirm the success of the one-hot encoding procedure on the test dataset
X_test_one_hot.head(3)

,nine_partition_label_benign dermal,nine_partition_label_benign epidermal,nine_partition_label_benign melanocyte,nine_partition_label_genodermatoses,nine_partition_label_inflammatory,nine_partition_label_malignant cutaneous lymphoma,nine_partition_label_malignant dermal,nine_partition_label_malignant epidermal,nine_partition_label_malignant melanoma
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


#### Check that every target class is present in all train, validation, and test datasets

In [27]:
# Define a function to check if all labels in validation and test sets are also present in the training set
def check_label_representativity(train, val, test):

    # Define unique labels in train, val and test
    unique_train = np.unique(train)
    unique_val = np.unique(val)
    unique_test = np.unique(test)
    
    # Iterate through each unique label in the validation set
    for label in unique_val:
        # Check if the current label from the validation set is not in the training set
        if label not in unique_train:
            # If a label in the validation set is not found in the training set, raise an exception
            raise Exception(f'Label {label} present in validation, but not in train')
            
    # Repeat the process for the test set
    for label in unique_test:
        # Check if the current label from the test set is not in the training set
        if label not in unique_train:
            # If a label in the test set is not found in the training set, raise an exception
            raise Exception(f'Label {label} present in test, but not in train') 
            
    # If no exceptions are raised, print a message indicating label representativity is ensured
    print('Label representativity is ensured across all datasets')

# Call the function with the labels of training, validation, and test sets as arguments
check_label_representativity(y_train, y_val, y_test)

Label representativity is ensured across all datasets


<a id='Data-Augmentation'></a>
### 2.5 Data Augmentation

In [28]:
# Data Augmentation setup
data_augmentation = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=0.2 # Zoom in by 20% maximum (randomly zoom images up to 20%)
)

# Initialize empty lists to store augmented data
augmented_images = []
augmented_numerical = [] 
augmented_one_hot = []  
augmented_labels = []

# Determine the number of samples to aim for each class after augmentation, to achieve balance
target_samples_per_class = int(len(y_train) * 0.03)

# Calculate how many samples each class needs based on the target
unique, counts = np.unique(y_train, return_counts=True)
class_needs = {class_label: max(0, target_samples_per_class - count) for class_label, count in zip(unique, counts)}

# Perform augmentation
for class_label, needed_samples in class_needs.items():
    if needed_samples <= 0:
        continue  # Skip classes that don't need augmentation

    # Indices of the current class in the dataset
    class_indices = np.where(y_train == class_label)[0]
    
    print(f'Augmenting class: {class_label}')

    # Augment data for this class
    for _ in range(needed_samples):
        idx = np.random.choice(class_indices)
        image_to_augment = X_train_image[idx].reshape((1,) + X_train_image[idx].shape)
        one_hot = X_train_one_hot.iloc[idx].values  # Corresponding one-hot features
        numerical = X_train_numerical[idx]          # Corresponding numerical feature
        # Perform augmentation
        it = data_augmentation.flow(image_to_augment, batch_size=1)
        augmented_image = next(it)[0]

        # Append the augmented data
        augmented_images.append(augmented_image)
        augmented_numerical.append(numerical)
        augmented_one_hot.append(one_hot)
        augmented_labels.append(class_label)

# Convert lists to numpy arrays
augmented_images = np.array(augmented_images)
augmented_one_hot = np.array(augmented_one_hot)
augmented_numerical = np.array(augmented_numerical)
augmented_labels = np.array(augmented_labels)

# Concatenate the original and augmented data
X_train_image_augmented = np.concatenate((X_train_image, augmented_images))
X_train_one_hot_augmented = np.concatenate((X_train_one_hot, augmented_one_hot), axis=0)
X_train_numerical_augmented = np.concatenate((X_train_numerical, augmented_numerical), axis=0)
y_train_augmented = np.concatenate((y_train, augmented_labels))

# Shuffle the augmented dataset
shuffle_indices = np.random.permutation(len(X_train_image_augmented))
X_train_image_augmented = X_train_image_augmented[shuffle_indices]
X_train_numerical_augmented = X_train_numerical_augmented[shuffle_indices]
X_train_one_hot_augmented = X_train_one_hot_augmented[shuffle_indices]
y_train_augmented = y_train_augmented[shuffle_indices]

Augmenting class: 0
Augmenting class: 1
Augmenting class: 2
Augmenting class: 3
Augmenting class: 4
Augmenting class: 5
Augmenting class: 6
Augmenting class: 7
Augmenting class: 8
Augmenting class: 9
Augmenting class: 10
Augmenting class: 11
Augmenting class: 12
Augmenting class: 13
Augmenting class: 14
Augmenting class: 15
Augmenting class: 16
Augmenting class: 17
Augmenting class: 18
Augmenting class: 19
Augmenting class: 20
Augmenting class: 21
Augmenting class: 22
Augmenting class: 23
Augmenting class: 24
Augmenting class: 25
Augmenting class: 26
Augmenting class: 27
Augmenting class: 28
Augmenting class: 29
Augmenting class: 30
Augmenting class: 31
Augmenting class: 32
Augmenting class: 33
Augmenting class: 34
Augmenting class: 35
Augmenting class: 36
Augmenting class: 37
Augmenting class: 38
Augmenting class: 39
Augmenting class: 40
Augmenting class: 41
Augmenting class: 42
Augmenting class: 43
Augmenting class: 44
Augmenting class: 45
Augmenting class: 46
Augmenting class: 47
Au

In [29]:
# Confirming the shape of the augmented training dataset
X_train_image_augmented.shape

(33420, 128, 128, 3)

In [30]:
label_series = pd.Series(y_train_augmented)  # Convert the labels into a Pandas Series

# Temporarily adjust Pandas settings to display all rows since the number of unique labels is relatively big (114)
pd.set_option('display.max_rows', None)

# Get the distribution of labels, sort by values in descending order to see the proportions
label_distribution = label_series.value_counts(normalize=True).sort_values(ascending=False)

print("Label Distribution After Data Augmentation:\n", label_distribution)

Label Distribution After Data Augmentation:
 86     0.011550
98     0.009874
77     0.008737
7      0.008737
47     0.008737
93     0.008737
113    0.008737
21     0.008737
65     0.008737
11     0.008737
106    0.008737
52     0.008737
39     0.008737
0      0.008737
30     0.008737
78     0.008737
69     0.008737
107    0.008737
46     0.008737
26     0.008737
82     0.008737
53     0.008737
50     0.008737
5      0.008737
62     0.008737
67     0.008737
103    0.008737
54     0.008737
27     0.008737
48     0.008737
36     0.008737
90     0.008737
81     0.008737
34     0.008737
101    0.008737
57     0.008737
1      0.008737
72     0.008737
105    0.008737
63     0.008737
24     0.008737
44     0.008737
95     0.008737
91     0.008737
41     0.008737
9      0.008737
35     0.008737
42     0.008737
49     0.008737
58     0.008737
22     0.008737
66     0.008737
29     0.008737
104    0.008737
64     0.008737
10     0.008737
96     0.008737
43     0.008737
75     0.008737
15     0.00

<a id='Modelling'></a>
# 3. Modelling

In [31]:
# Delete extra datasets to save memory
del X_temp_image
del X_temp_numerical
del X_temp_categorical
del y_temp
del images

<a id='Base-Model'></a>
### 3.1 Base Model

In [39]:
def create_base_model():
    

    # Base model architecture
    model = Sequential([
        Conv2D(10, kernel_size=3, activation='relu', input_shape=(128, 128, 3)), 
        Flatten(),
        Dense(114, activation='softmax')
    ])

    optimizer = Adagrad(learning_rate=0.00583208197094374)

    # Compile the model
    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

model = create_base_model()

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=0.0001, verbose=1)

# Fit the model
history = model.fit(
    x=X_train_image, y=y_train,
    validation_data=(X_val_image, y_val),
    epochs=20, verbose=1,
    batch_size=32,
    callbacks=[early_stopping, reduce_lr],
)

# Evaluation
accuracy = model.evaluate(X_test_image, y_test, verbose=0)

scores = model.predict(X_test_image)
y_pred = np.argmax(scores, axis=1)
test_accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')


print(f'Accuracy: {accuracy}')
print(f'Test Accuracy: {test_accuracy}')
print(f'F1 Score: {f1}')


Epoch 1/20


305/305 [==============================] - 21s 69ms/step - loss: 4.5623 - accuracy: 0.0437 - val_loss: 4.4403 - val_accuracy: 0.0484 - lr: 0.0058
Epoch 2/20
305/305 [==============================] - 22s 71ms/step - loss: 4.3607 - accuracy: 0.0577 - val_loss: 4.3919 - val_accuracy: 0.0530 - lr: 0.0058
Epoch 3/20
305/305 [==============================] - 24s 79ms/step - loss: 4.2432 - accuracy: 0.0713 - val_loss: 4.3848 - val_accuracy: 0.0443 - lr: 0.0058
Epoch 4/20
305/305 [==============================] - 24s 77ms/step - loss: 4.1200 - accuracy: 0.0917 - val_loss: 4.2798 - val_accuracy: 0.0619 - lr: 0.0058
Epoch 5/20
305/305 [==============================] - 23s 75ms/step - loss: 3.9902 - accuracy: 0.1084 - val_loss: 4.2969 - val_accuracy: 0.0674 - lr: 0.0058
Epoch 6/20
305/305 [==============================] - 22s 72ms/step - loss: 3.8725 - accuracy: 0.1279 - val_loss: 4.2575 - val_accuracy: 0.0656 - lr: 0.0058
Epoch 7/20
305/305 [==============================] - 24s 80ms/step -

<a id='Regularized-Model'></a>
### 3.2 Regularized Model

In [33]:
def create_model():

    # Regularized model architecture
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(114, activation='softmax')
    ])
    optimizer = Adagrad(learning_rate=0.004901409801938672)
    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

model = create_model()
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=0.0001, verbose=1)

# Fit the model
history = model.fit(
    x=X_train_image, y=y_train,
    validation_data=(X_val_image, y_val),
    epochs=20, verbose=1,
    batch_size=64,
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/20


2024-04-28 14:42:51.937898: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


153/153 [==============================] - 81s 525ms/step - loss: 4.9315 - accuracy: 0.0319 - val_loss: 5.1737 - val_accuracy: 0.0182 - lr: 0.0049
Epoch 2/20
153/153 [==============================] - 84s 550ms/step - loss: 4.4438 - accuracy: 0.0675 - val_loss: 4.8247 - val_accuracy: 0.0345 - lr: 0.0049
Epoch 3/20
153/153 [==============================] - 86s 561ms/step - loss: 4.1327 - accuracy: 0.1045 - val_loss: 4.5139 - val_accuracy: 0.0511 - lr: 0.0049
Epoch 4/20
153/153 [==============================] - 87s 568ms/step - loss: 3.8055 - accuracy: 0.1530 - val_loss: 4.4879 - val_accuracy: 0.0758 - lr: 0.0049
Epoch 5/20
153/153 [==============================] - 88s 576ms/step - loss: 3.4199 - accuracy: 0.2140 - val_loss: 4.1798 - val_accuracy: 0.1026 - lr: 0.0049
Epoch 6/20
153/153 [==============================] - 88s 578ms/step - loss: 2.9899 - accuracy: 0.2936 - val_loss: 4.1391 - val_accuracy: 0.1170 - lr: 0.0049
Epoch 7/20
153/153 [==============================] - 90s 591ms

In [34]:
# Evaluation
accuracy = model.evaluate(X_val_image, y_val, verbose=0)
scores = model.predict(X_test_image)
y_pred = np.argmax(scores, axis=1)
test_accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')


print(f'Accuracy: {accuracy}')
print(f'Test Accuracy: {test_accuracy}')
print(f'F1 Score: {f1}')

102/102 [==============================] - 7s 70ms/step
Accuracy: [4.0721893310546875, 0.15275639295578003]
Test Accuracy: 0.16410098522167488
F1 Score: 0.15003962720098593


<a id='Model-With-Functional-API-Augmented'></a>
### 3.4 Model With Functional API With Data Augmentation

In [35]:
def create_model():
    
    # Inputs
    image_input = Input(shape=(128, 128, 3), name='image_input')
    numerical_input = Input(shape=(1,), name='numerical_input')
    one_hot_input = Input(shape=(9,), name='one_hot_input')

    # Image processing branch
    x = Conv2D(32, (3, 3), activation='relu')(image_input)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(0.33846602232096573)(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(0.33846602232096573)(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(0.33846602232096573)(x)
    x = Flatten()(x)

    # Numerical data processing branch
    y = Dense(64, activation='relu')(numerical_input)
    y = BatchNormalization()(y)

    # One-hot encoded data processing branch
    z = Dense(64, activation='relu')(one_hot_input)
    z = BatchNormalization()(z)

    # Concatenate all layers
    combined = concatenate([x, y, z])
    combined = Dense(512, activation='relu')(combined)
    combined = BatchNormalization()(combined)
    combined = Dropout(0.668798647763459)(combined)
    output = Dense(114, activation='softmax')(combined)

        
    optimizer = Adam(learning_rate=0.004253738267116145)

    model = Model(inputs=[image_input, numerical_input, one_hot_input], outputs=output)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model


model = create_model()

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=0.0001, verbose=1)

# Fit the model
history = model.fit(
    [X_train_image_augmented, X_train_numerical_augmented, X_train_one_hot_augmented], y_train_augmented,
    validation_data=([X_val_image, X_val_numerical, X_val_one_hot], y_val),
    epochs=20, verbose=1,
    batch_size=64,
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/20
523/523 [==============================] - 318s 606ms/step - loss: 4.2676 - accuracy: 0.1125 - val_loss: 3.3266 - val_accuracy: 0.1518 - lr: 0.0043
Epoch 2/20
523/523 [==============================] - 319s 609ms/step - loss: 2.9674 - accuracy: 0.2320 - val_loss: 3.2526 - val_accuracy: 0.1534 - lr: 0.0043
Epoch 3/20
523/523 [==============================] - 308s 590ms/step - loss: 2.7182 - accuracy: 0.2900 - val_loss: 3.2005 - val_accuracy: 0.1931 - lr: 0.0043
Epoch 4/20
523/523 [==============================] - 311s 595ms/step - loss: 2.5111 - accuracy: 0.3360 - val_loss: 3.4055 - val_accuracy: 0.1552 - lr: 0.0043
Epoch 5/20
523/523 [==============================] - 308s 590ms/step - loss: 2.2977 - accuracy: 0.3911 - val_loss: 3.2508 - val_accuracy: 0.2026 - lr: 0.0043
Epoch 6/20
523/523 [==============================] - 312s 596ms/step - loss: 2.0738 - accuracy: 0.4485 - val_loss: 2.9493 - val_accuracy: 0.2796 - lr: 0.0043
Epoch 7/20
523/523 [==========================

In [38]:
# Evaluate the model
accuracy = model.evaluate([X_test_image, X_test_numerical, X_test_one_hot], y_test, verbose=0)
scores = model.predict([X_test_image, X_test_numerical, X_test_one_hot])
y_pred = np.argmax(scores, axis=1)
test_accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')


print(f'Accuracy: {accuracy}')
print(f'Test Accuracy: {test_accuracy}')
print(f'F1 Score: {f1}')

102/102 [==============================] - 7s 70ms/step
Accuracy: [2.812701940536499, 0.3158867061138153]
Test Accuracy: 0.3158866995073892
F1 Score: 0.3135323235002952


<a id='Transfer-Learning-With-Functional-API'></a>
### 3.6 Transfer Learning With Functional API With Data Augmentation

#### **Dense Net**

In [44]:
def create_model(dropout_rate, num_units, num_units_one_hot, optimizer, lr, dropout_rate_combined, loss, metrics):
    
    # Inputs
    image_input = Input(shape=(128, 128, 3), name='image_input')
    numerical_input = Input(shape=(1,), name='numerical_input')
    one_hot_input = Input(shape=(9,), name='one_hot_input')

    # DenseNet201-based image processing branch
    base_model = DenseNet201(include_top=False, weights="imagenet", input_tensor=image_input)

    # Only train the last few layers (specifically, layers after 'conv5_block30_0_bn')
    flag = False
    for layer in base_model.layers:
        if layer.name == 'conv5_block30_0_bn' or flag:
            layer.trainable = True
            flag = True
        else:
            layer.trainable = False

    x = GlobalAveragePooling2D()(base_model.output)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)

    # Numerical data processing branch
    y = Dense(num_units)(numerical_input)
    y = BatchNormalization()(y)

    # One-hot encoded data processing branch
    z = Dense(num_units_one_hot)(one_hot_input)
    z = BatchNormalization()(z)

    # Concatenate all layers
    combined = concatenate([x, y, z])
    combined = Dense(512, activation='relu')(combined)
    combined = BatchNormalization()(combined)
    combined = Dropout(dropout_rate_combined)(combined)
    output = Dense(114, activation='softmax')(combined)


    lr = lr
    optimizer = optimizer

    model = Model(inputs=[image_input, numerical_input, one_hot_input], outputs=output)
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model

model = create_model(dropout_rate = 0.44015986503365345, num_units = 64, num_units_one_hot = 16, dropout_rate_combined = 0.5989644076124379, optimizer = 'SGD', lr = 0.003068882482985734, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=0.0001, verbose=1)

# Fit the model
history = model.fit(
    [X_train_image_augmented, X_train_numerical_augmented, X_train_one_hot_augmented], y_train_augmented,
    validation_data=([X_val_image, X_val_numerical, X_val_one_hot], y_val),
    epochs=20, batch_size=64,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)



Epoch 1/20
523/523 [==============================] - 629s 1s/step - loss: 5.0447 - accuracy: 0.0750 - val_loss: 3.5003 - val_accuracy: 0.2147 - lr: 0.0100
Epoch 2/20
523/523 [==============================] - 641s 1s/step - loss: 3.4901 - accuracy: 0.2228 - val_loss: 2.8702 - val_accuracy: 0.3107 - lr: 0.0100
Epoch 3/20
523/523 [==============================] - 647s 1s/step - loss: 2.8201 - accuracy: 0.3185 - val_loss: 2.6291 - val_accuracy: 0.3486 - lr: 0.0100
Epoch 4/20
523/523 [==============================] - 675s 1s/step - loss: 2.4489 - accuracy: 0.3876 - val_loss: 2.4982 - val_accuracy: 0.3699 - lr: 0.0100
Epoch 5/20
523/523 [==============================] - 693s 1s/step - loss: 2.1969 - accuracy: 0.4355 - val_loss: 2.4153 - val_accuracy: 0.3847 - lr: 0.0100
Epoch 6/20
523/523 [==============================] - 722s 1s/step - loss: 2.0023 - accuracy: 0.4738 - val_loss: 2.3558 - val_accuracy: 0.3936 - lr: 0.0100
Epoch 7/20
523/523 [==============================] - 686s 1s/st

In [47]:
scores = model.predict([X_test_image, X_test_numerical, X_test_one_hot])
y_pred = np.argmax(scores, axis=1)
test_accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')


print(f'Accuracy: {accuracy}')
print(f'Test Accuracy: {test_accuracy}')
print(f'F1 Score: {f1}')

102/102 [==============================] - 71s 684ms/step
Accuracy: [4.172484397888184, 0.10621920973062515]
Test Accuracy: 0.4362684729064039
F1 Score: 0.4328489990863967
